In [3]:
import pennylane as qml
import numpy as np
from divi.backends import ParallelSimulator
from divi.qprog import GenericLayerAnsatz, HartreeFockAnsatz, UCCSDAnsatz, VQE
from divi.qprog.optimizers import ScipyOptimizer, ScipyMethod
from divi.qprog.workflows import VQEHyperparameterSweep, MoleculeTransformer

# Different models: 

# 1. First, we test the most simple ansatz with only one rotation
simple = GenericLayerAnsatz(
    gate_sequence=[qml.RY], 
    entangler=qml.CNOT, 
    entangling_layout="linear" 
    )
# 2. Secondly, we test the more complex ansatz with one y- and one z- rotation
balanced = GenericLayerAnsatz(
    gate_sequence=[qml.RY, qml.RZ], 
    entangler=qml.CNOT, 
    entangling_layout="linear"
    )
# 3. Thirdly, we test the more complex ansatz with one y- and one z- rotation and entangling: all-to-all
expensive = GenericLayerAnsatz(
    gate_sequence=[qml.RY, qml.RZ], 
    entangler=qml.CNOT, 
    entangling_layout="all_to_all"
    )
# 3. Thirdly, we test the HF-ansatz
hf = HartreeFockAnsatz()
# 4. Lastly, we benchmark the results agains the accurate UCCSD - model
uccsd = UCCSDAnsatz()



# Define optimizer
optimizer = ScipyOptimizer(method=ScipyMethod.L_BFGS_B)


# Benchmarking 
# Define the active space for a 12-qubit simulation
# This freezes core 1s electrons and focuses on valence electrons
active_electrons = 8
active_orbitals = 6

# The two degenerate configurations of Ammonia (coordinates)
nh3_config1_coords = np.array(
    [
        (0, 0, 0), # N  
        (1.01, 0, 0), # H₁  
        (-0.5, 0.87, 0), # H₂  
        (-0.5, -0.87, 0) # H₃
    ]  
)

nh3_config2_coords = np.array(
    [
        (0, 0, 0),  # N (inverted)
        (-1.01, 0, 0),  # H₁
        (0.5, -0.87, 0),  # H₂
        (0.5, 0.87, 0),  # H₃
    ]
)

# Create molecule objects
nh3_molecule1 = qml.qchem.Molecule(
    symbols=["N", "H", "H", "H"],
    coordinates=nh3_config1_coords
)

nh3_molecule2 = qml.qchem.Molecule(
    symbols=["N", "H", "H", "H"],
    coordinates=nh3_config2_coords
)

# Build Hamiltonians with active space parameters
hamiltonian1, qubits = qml.qchem.molecular_hamiltonian(
    nh3_molecule1,
    active_electrons=active_electrons,
    active_orbitals=active_orbitals,
)

hamiltonian2, qubits = qml.qchem.molecular_hamiltonian(
    nh3_molecule2,
    active_electrons=active_electrons,
    active_orbitals=active_orbitals,
)


In [3]:
n_qubits=12
H1 = qml.matrix(hamiltonian1, wire_order=range(n_qubits))
E, V = np.linalg.eigh(H1)

In [4]:
E[0]

-53.114040480575156

In [5]:
n_qubits=12
H2 = qml.matrix(hamiltonian2, wire_order=range(n_qubits))
E, V = np.linalg.eigh(H2)

In [6]:
E[0]

-53.114040480575156

In [1]:
import pennylane as qml
from pennylane import numpy as np

electrons = 2
n_qubits = 4
hf = qml.qchem.hf_state(electrons, n_qubits)   # e.g. [1, 1, 0, 0]

#dev = qml.device("default.qubit", wires=n_qubits)

#@qml.qnode(dev)
def circuit(params, ansatz):
    # 1) Prepare Hartree–Fock reference
    qml.BasisState(hf, wires=range(n_qubits))
    # 2) Put your ansatz "on top"
    ansatz.build(params, n_qubits=n_qubits, n_layers=1)
    return qml.expval(qml.Z(0))


In [ ]:
import pennylane as qml
from pennylane import qchem
from typing import Any

class HFLayerAnsatz(GenericLayerAnsatz):
    """
    GenericLayerAnsatz on top of a Hartree-Fock reference state.

    Usage:
        ansatz = HFLayerAnsatz(
            gate_sequence=[qml.RY, qml.RZ],
            entangler=qml.CNOT,
            entangling_layout="linear",
        )

        # later, when building:
        ops = ansatz.build(
            params,
            n_qubits=n_qubits,
            n_layers=n_layers,
            n_electrons=n_electrons,  # or hf_state=...
        )
    """

    def build(
        self,
        params: Any,
        n_qubits: int,
        n_layers: int,
        **kwargs: Any,
    ) -> list[qml.operation.Operator]:
        # Option A: user directly passes a bitstring/array as hf_state
        hf_state = kwargs.pop("hf_state", None)

        # Option B: derive HF state from number of electrons
        if hf_state is None:
            n_electrons = kwargs.get("n_electrons", None)
            if n_electrons is None:
                raise ValueError(
                    "HFLayerAnsatz.build requires either `hf_state` or `n_electrons` "
                    "in kwargs."
                )
            hf_state = qchem.hf_state(n_electrons, n_qubits)

        wires = list(range(n_qubits))

        # 1) HF preparation as the very first operation
        operations: list[qml.operation.Operator] = [
            qml.BasisState(hf_state, wires=wires)
        ]

        # 2) All the usual layers from GenericLayerAnsatz
        layer_ops = super().build(params, n_qubits=n_qubits, n_layers=n_layers, **kwargs)

        return operations + layer_ops


In [5]:
simple_hf = HFLayerAnsatz(
    gate_sequence=[qml.RY],
    entangler=qml.CNOT,
    entangling_layout="linear",
)

balanced_hf = HFLayerAnsatz(
    gate_sequence=[qml.RY, qml.RZ],
    entangler=qml.CNOT,
    entangling_layout="linear",
)

expensive_hf = HFLayerAnsatz(
    gate_sequence=[qml.RY, qml.RZ],
    entangler=qml.CNOT,
    entangling_layout="all_to_all",
)


In [8]:
expensive_hf.gate_sequence

[pennylane.ops.qubit.parametric_ops_single_qubit.RY,
 pennylane.ops.qubit.parametric_ops_single_qubit.RZ]

In [ ]:
circ = circuit()

In [ ]:
ansatze = [hf, balanced, uccsd, simple, expensive]
ansatze = [balanced, simple]  # add more if you want

optimizer = ScipyOptimizer(method=ScipyMethod.L_BFGS_B)
n_layers = 1
# Sweep object for geometry 1
energies = np.zeros((len(ansatze), 2))
circuit_counts = np.zeros((len(ansatze), 2))
backend = ParallelSimulator(shots=5000)
for i, ansatz in enumerate(ansatze):
    vqe1 = VQE(hamiltonian1, 
               n_layers=n_layers, 
               ansatz=ansatz, 
               max_iterations=50, 
               backend=backend)
    
    vqe1.run()
    energies[i, 0] = vqe1.best_loss
    circuit_counts[i, 0] = vqe1.total_circuit_count



    vqe2 = VQE(hamiltonian2, 
            n_electrons=8, 
            n_layers=n_layers, 
            ansatz=ansatz, 
            max_iterations=50, 
            backend=backend)
    
    vqe2.run()
    energies[i, 1] = vqe2.best_loss
    circuit_counts[i, 1] = vqe2.total_circuit_count


# Run sweeps for both geometries
print(energies)
print(circuit_counts)

[11/29/25 18:07:33] INFO     divi.reporting - INFO - Finished Setup

Output()

[11/29/25 18:08:19] INFO     divi.reporting - INFO - Finished Iteration #1

Output()

[11/29/25 18:09:03] INFO     divi.reporting - INFO - Finished Iteration #2

...

Output()

[11/29/25 18:23:15] INFO     divi.reporting - INFO - Finished Iteration #22

Output()

[11/29/25 18:24:02] INFO     divi.reporting - INFO - Finished Iteration #23

Output()